# Deep Leraning Base Model

The goal of this notebook is to build a full deep learning pipeline with the base DL model.

## Imports and Data Load

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from dispatcher.data.join import JoinTables
from dispatcher.transformers.encoders import TimeFeaturesEncoder
from sklearn import set_config; set_config(display='diagram')
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier

In [3]:
X_train, X_test, y_train, y_test = JoinTables().join_tables()

/Users/christianklaus/code/christianklausML/dispatcher-project/dispatcher/data/ticket.py:24: DtypeWarning: Columns (4,12,22,24,26) have mixed types.Specify dtype option on import or set low_memory=False.
  shipment_order = RawData.get_table_data('shipment', local=True,


ticket features loaded
order features loaded.
carrier features loaded.


/Users/christianklaus/code/christianklausML/dispatcher-project/dispatcher/data/shipment.py:10: DtypeWarning: Columns (4,12,14) have mixed types.Specify dtype option on import or set low_memory=False.
  clean_shipment = RawData.get_table_data('shipment', local=True, clean=True)


warehouse features loaded
Tables joined.
NAs filled. (─‿‿─)
Number of rows: 1403405
                   percent_missing
WAREHOUSE_ID                     0
RELATION_DISTANCE                0
SCR_HRS                          0
SCR_MIN                          0
PPU_DOW                          0
0.0% of rows dropped.
train test split ready. ( ◑‿◑)


In [4]:
X_train.head()

,WAREHOUSE_ID,RELATION_DISTANCE,CARRIER_COMPANY_ID,SHOP_ID,DESTINATION_ZIP_CODE,CUSTOMER_ADDRESS_ZIP_CODE,CUSTOMER_ADDRESS_COUNTRY_ID,OC_MIN,OC_HRS,OC_DOW,OC_MONTH,PPU_MIN,PPU_HRS,PPU_DOW,SCR_MIN,SCR_HRS,SCR_DOW
1466960,604,896.317844,76,405,CV8 3ED,NE10 0AZ,235,16,22,2,3,16,22,2,16,21,2
60826,405,245.468977,2,14,1099,1600,22,2,17,0,3,0,17,2,45,9,3
1107281,673,360.317753,28,465,21241,58212,215,50,21,1,3,50,21,1,55,23,1
1318912,310,827.111602,13,196,2333,9073,15,3,20,1,3,3,20,1,17,20,1
594668,613,689.355960,4,196,20018,12042,110,10,17,1,3,10,17,1,22,17,1


In [5]:
X_train.dtypes

WAREHOUSE_ID                     int64
RELATION_DISTANCE              float64
CARRIER_COMPANY_ID               int64
SHOP_ID                          int64
DESTINATION_ZIP_CODE            string
CUSTOMER_ADDRESS_ZIP_CODE       string
CUSTOMER_ADDRESS_COUNTRY_ID      int64
OC_MIN                           int64
OC_HRS                           int64
OC_DOW                           int64
OC_MONTH                         int64
PPU_MIN                          int64
PPU_HRS                          int64
PPU_DOW                          int64
SCR_MIN                          int64
SCR_HRS                          int64
SCR_DOW                          int64
dtype: object

In [6]:
y_train.shape

(1122719, 5)

## Pipeline Building


In [11]:
def get_model():
    model = Sequential()
    model.add(layers.Dense(50, activation='relu', input_dim=200))
    model.add(layers.Dense(25, activation='relu'))
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dense(5, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', 
                  metrics=['accuracy']) 
    return model

es = EarlyStopping(patience=10)
clf = KerasClassifier(get_model, epochs=10, batch_size=32,
                     verbose=1, callbacks= [es], validation_split=0.3)

In [12]:
num_transformer = make_pipeline(StandardScaler())
num_col = ['RELATION_DISTANCE']

cat_transformer = OneHotEncoder(sparse=False)
cat_col = ['WAREHOUSE_ID','CARRIER_COMPANY_ID',
          'PPU_HRS','PPU_DOW',
          'SCR_HRS','SCR_DOW']

preproc = make_column_transformer(
    (num_transformer, num_col),
    (cat_transformer, cat_col),
    remainder='drop')

pipeline = make_pipeline(preproc, clf)
pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  ['RELATION_DISTANCE']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(sparse=False),
                                                  ['WAREHOUSE_ID',
                                                   'CARRIER_COMPANY_ID',
                                                   'PPU_HRS', 'PPU_DOW',
                                                   'SCR_HRS', 'SCR_DOW'])])),
                ('kerasclassifier',
                 KerasClassifier(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x15f71ba30>], epochs=10, model=<function get_model at 0x15f6fbca0>, validation_split=0.3))])

In [13]:
pipeline.fit(X_train, y_train)

Epoch 1/10
24560/24560 [==============================] - 60s 2ms/step - loss: 0.4993 - accuracy: 0.7927 - val_loss: 0.4418 - val_accuracy: 0.8106
Epoch 2/10
24560/24560 [==============================] - 59s 2ms/step - loss: 0.4368 - accuracy: 0.8157 - val_loss: 0.4266 - val_accuracy: 0.8197
Epoch 3/10
24560/24560 [==============================] - 49s 2ms/step - loss: 0.4240 - accuracy: 0.8202 - val_loss: 0.4193 - val_accuracy: 0.8228
Epoch 4/10
24560/24560 [==============================] - 49s 2ms/step - loss: 0.4172 - accuracy: 0.8222 - val_loss: 0.4205 - val_accuracy: 0.8223
Epoch 5/10
24560/24560 [==============================] - 50s 2ms/step - loss: 0.4126 - accuracy: 0.8239 - val_loss: 0.4110 - val_accuracy: 0.8243
Epoch 6/10
24560/24560 [==============================] - 48s 2ms/step - loss: 0.4094 - accuracy: 0.8247 - val_loss: 0.4086 - val_accuracy: 0.8225
Epoch 7/10
24560/24560 [==============================] - 49s 2ms/step - loss: 0.4067 - accuracy: 0.8260 - val_loss: 0

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  ['RELATION_DISTANCE']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(sparse=False),
                                                  ['WAREHOUSE_ID',
                                                   'CARRIER_COMPANY_ID',
                                                   'PPU_HRS', 'PPU_DOW',
                                                   'SCR_HRS', 'SCR_DOW'])])),
                ('kerasclassifier',
                 KerasClassifier(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x15f71ba30>], epochs=10, model=<function get_model at 0x15f6fbca0>, validation_split=0.3))])

In [ ]:
score = pipeline.score(X_test, y_test)
score

In [ ]:
X_train